## Page 09 - Valuation (section)

We now move from ESG context to the core valuation toolkit.

<details>
<summary>Slide screenshot (PDF page 9)</summary>

![](pic/0209_lecture6_firm_valuation/page-09.png)

</details>


In [1]:
# Setup: imports + local data paths
from pathlib import Path
import os

import numpy as np
import pandas as pd

# Optional online fetches (akshare) are disabled by default.
RUN_ONLINE = False
RUN_ONLINE = RUN_ONLINE or (os.environ.get('RUN_ONLINE', '0') == '1')

# Make paths robust whether Jupyter is launched from repo root or from `materials/`.
DATA_DIR = Path('data') if Path('data').exists() else Path('materials/data')
SLIDE_DIR = (
    Path('pic') / '0209_lecture6_firm_valuation'
    if (Path('pic') / '0209_lecture6_firm_valuation').exists()
    else Path('materials/pic/0209_lecture6_firm_valuation')
)

if not DATA_DIR.exists():
    raise FileNotFoundError(f'Cannot find data directory: {DATA_DIR.resolve()}')
if not SLIDE_DIR.exists():
    raise FileNotFoundError(f'Cannot find slide directory: {SLIDE_DIR.resolve()}')

# Load Tesla statements used in the DCF exercise.
income = pd.read_excel(DATA_DIR / 'tesla_income_stmt.xlsx', index_col=0)
cashflow = pd.read_excel(DATA_DIR / 'tesla_cash_flow.xlsx', index_col=0)
balance = pd.read_excel(DATA_DIR / 'tesla_balance_sheet.xlsx', index_col=0)

base_year = income.index.max()
prev_year = income.index.sort_values()[-2]
base_year


Timestamp('2025-12-31 00:00:00')

## Page 10 - What is valuation? Firm value vs equity value

**Valuation** estimates the economic worth of a firm (or an asset) based on fundamentals such as expected future
cash flows.

- **Market price** reflects current trading conditions.
- **Intrinsic value** reflects underlying economic drivers.

**Key identity**

`Equity Value = Enterprise Value (Firm Value) - Net Debt`

| Concept | What it represents |
|---|---|
| Enterprise value (firm value) | Market value of all capital invested in the business (equity + net debt) |
| Equity value | Market value of shareholders' equity (shares outstanding * stock price) |

<details>
<summary>Slide screenshot (PDF page 10)</summary>

![](pic/0209_lecture6_firm_valuation/page-10.png)

</details>


## Page 11 - Reasons for Performing a Valuation

Valuation is used in several settings:

| Category | Purpose | Typical question |
|---|---|---|
| Transaction & market decisions | Buying/selling a business | What is a fair transaction price? |
| Transaction & market decisions | Securities investing | Is the security mispriced vs intrinsic value / peers? |
| Strategic decisions | Strategic planning | Which projects/acquisitions increase firm value? |
| Strategic decisions | Capital financing | Can the firm generate cash flows to service debt? |
| Regulatory / accounting / compliance | Tax and financial reporting | What is fair value for reporting and compliance? |

<details>
<summary>Slide screenshot (PDF page 11)</summary>

![](pic/0209_lecture6_firm_valuation/page-11.png)

</details>


## Page 12 - Core Valuation Approaches

Four common approaches:

1. **Discounted Cash Flow (DCF)**
   - Intrinsic (fundamentals-based) valuation
   - Values the firm as the present value of expected free cash flows

2. **Public Trading Comparables**
   - Market approach using trading multiples of similar publicly listed firms
   - Often used for IPO pricing and as a cross-check for DCF

3. **Precedent Transactions**
   - Market approach using valuation multiples observed in comparable M&A deals
   - Used in M&A negotiations and fairness opinions

4. **Leveraged Buyout (LBO)**
   - Values the deal from a financial sponsor perspective
   - Focuses on cash flows, debt capacity, and equity IRR

*Source in slide: J.P. Morgan.*

<details>
<summary>Slide screenshot (PDF page 12)</summary>

![](pic/0209_lecture6_firm_valuation/page-12.png)

</details>


## Page 13 - Valuation Framework

A practical workflow:

1. **Import and review historical financials**
   - Understand recent performance and establish a base year.

2. **Set forecasting assumptions**
   - Revenue: growth, pricing, market share, industry cycle
   - Investment: capex plans, capital intensity
   - Financing: debt/equity needs and funding constraints

3. **Project financial statements**
   - Operating, investing, and financing activities are linked.

4. **Run intrinsic valuation (DCF)**

5. **Sensitivity analysis**
   - How does intrinsic value change with assumptions (e.g., WACC, terminal growth)?

6. **Cross-check with comparable-company multiples**

*Source in slide: McKinsey.*

<details>
<summary>Slide screenshot (PDF page 13)</summary>

![](pic/0209_lecture6_firm_valuation/page-13.png)

</details>


## Page 14 - Discounted Cash Flow Analysis (DCF)

DCF values the firm as:

- present value of forecast **free cash flows (FCF)** during an explicit projection period, plus
- a **terminal value** capturing cash flows beyond the forecast horizon.

Typical steps:
1. Forecast annual FCF.
2. Estimate an appropriate discount rate (WACC).
3. Estimate terminal value (growing-perpetuity or exit multiple).

The next cell provides clean formulas you can reuse.

<details>
<summary>Slide screenshot (PDF page 14)</summary>

![](pic/0209_lecture6_firm_valuation/page-14.png)

</details>


### DCF recap (clean formulas)

We'll use the standard operating FCF definition (firm value):

$$
FCF_t = EBIT_t(1-T) + D\&A_t - CapEx_t - \Delta NWC_t
$$

and discount with WACC:

$$
V = \sum_{t=1}^{n} \frac{FCF_t}{(1+WACC)^t} + \frac{TV_n}{(1+WACC)^n}
$$


In [2]:
def dcf_pv(cashflows, wacc):
    cashflows = np.asarray(cashflows, dtype=float)
    t = np.arange(1, len(cashflows) + 1)
    return (cashflows / (1 + wacc) ** t).sum()

def terminal_value_growing_perpetuity(fcf_t, wacc, g):
    if g >= wacc:
        raise ValueError('g must be smaller than WACC for a stable perpetuity.')
    return fcf_t * (1 + g) / (wacc - g)


## Page 15 - Projecting the Free Cash Flows

When forecasting FCF, the slide highlights common drivers:

- **Industry outlook:** growth, supply vs demand, major risks/opportunities
- **Competitive position:** pricing power, market share, cost structure
- **Ongoing reinvestment needs:** working capital, required capex, discretionary investments
- **Expansion opportunities:** new product lines, greenfield expansion, development costs, economies of scale
- **Sources of information:** company reports, competitor reports, equity research, industry data

<details>
<summary>Slide screenshot (PDF page 15)</summary>

![](pic/0209_lecture6_firm_valuation/page-15.png)

</details>


## Page 16 - Estimate the Discount Rate: Cost of Equity

DCF requires a discount rate that reflects risk. The standard decomposition is:

$$
WACC = \frac{E}{V} r_e + \frac{D}{V} r_d (1-T)
$$

The **cost of equity** is commonly estimated using CAPM:

$$
r_e = r_f + \beta (ERP)
$$

Where:
- $r_f$: long-term risk-free rate (often a government bond yield)
- $\beta$: sensitivity to market returns (systematic risk)
- $ERP$: equity risk premium

<details>
<summary>Slide screenshot (PDF page 16)</summary>

![](pic/0209_lecture6_firm_valuation/page-16.png)

</details>


## Page 17 - Estimate the Discount Rate: Cost of Debt

A simple way to think about the cost of debt:

$$
r_d \approx r_f + \text{debt spread}
$$

The spread can be estimated using:
- **Bond yield spread:** corporate bond yield minus government bond yield (same maturity)
- **Credit-rating-implied spread:** when bond data are unavailable
- **Loan markup:** lender-specific interest premium

In WACC we typically use the **after-tax** cost of debt: $r_d(1-T)$.

<details>
<summary>Slide screenshot (PDF page 17)</summary>

![](pic/0209_lecture6_firm_valuation/page-17.png)

</details>


## Page 18 - Estimate the Terminal Value

Two common approaches:

1. **Growing perpetuity (steady state)**

$$
TV_n = \frac{FCF_{n+1}}{WACC - g}
$$

2. **Exit multiple (not yet steady state)**
- Estimate terminal-year EBITDA/EBIT (or another metric)
- Multiply by a peer multiple to obtain $TV_n$

Valuation takeaway: terminal value can dominate DCF results, so assumptions about $g$ and the steady state are
high-leverage.

<details>
<summary>Slide screenshot (PDF page 18)</summary>

![](pic/0209_lecture6_firm_valuation/page-18.png)

</details>


## Page 19 - Intrinsic Valuation of Tesla

The slide lays out a Tesla DCF setup:

- **Primary method:** DCF
- **Cross-check:** relative valuation (trading multiples)
- **Data source:** Yahoo Finance (or other market-data providers)

Build the base year from historical statements:
- Income statement: revenue, EBIT, taxes
- Cash flow statement: D&A, capex
- Balance sheet: operating working capital, cash, debt

Then compute historical ratios (margins and scaling relationships) to guide forecasting assumptions.

<details>
<summary>Slide screenshot (PDF page 19)</summary>

![](pic/0209_lecture6_firm_valuation/page-19.png)

</details>


## Page 20 - Project Future Cash Flows

A common forecasting approach is to:

- start with revenue growth expectations (analyst inputs or your own scenario)
- translate profitability expectations into an EBIT margin path
- scale D&A and capex with revenue (with convergence logic in the steady state)
- estimate $\Delta NWC$ as a function of incremental revenue

Finally compute operating FCF each year:

$$
FCF_t = EBIT_t(1-T) + D\&A_t - CapEx_t - \Delta NWC_t
$$

<details>
<summary>Slide screenshot (PDF page 20)</summary>

![](pic/0209_lecture6_firm_valuation/page-20.png)

</details>


## Page 21 - Estimate the Discount Rate (WACC)

For Tesla, the slide suggests:

1. **Cost of equity (CAPM)**
   - $r_f$: U.S. 10-year Treasury yield (proxy)
   - $\beta$: market beta
   - $ERP$: equity risk premium

2. **Capital structure**
   - Use market capitalization and total debt (market-value weights)

3. **Cost of debt**
   - Use yields (YTM) on outstanding corporate bonds, or a rating-implied spread

In the Python section, we use teaching assumptions to keep the workflow transparent.

<details>
<summary>Slide screenshot (PDF page 21)</summary>

![](pic/0209_lecture6_firm_valuation/page-21.png)

</details>


## Page 22 - Estimate Terminal Value

The slide motivates terminal-value assumptions using reinvestment logic.

A useful identity linking payout policy and growth is:

$$
g = b \cdot ROE
$$

where $b$ is the retention ratio ($b = 1 - \text{payout ratio}$).

For Tesla, the slide notes:
- payout ratio near `0%` implies continued reinvestment
- Tesla is in a transition phase rather than a fully steady state
- assume a steady state by the end of the explicit forecast period (e.g., 2030)
- pick terminal $g$ consistent with long-run nominal GDP growth

<details>
<summary>Slide screenshot (PDF page 22)</summary>

![](pic/0209_lecture6_firm_valuation/page-22.png)

</details>


### Python practice: a compact Tesla DCF + carbon adjustment

We now implement a *teaching* DCF model that follows the slide logic:

- forecast revenue, EBIT margin, D&A/revenue, capex/revenue
- estimate operating working capital from incremental revenue
- discount by WACC and compute a growing-perpetuity terminal value

Then we connect carbon footprint -> valuation by subtracting an after-tax **carbon cost** scenario.


In [3]:
# Base-year ratios for forecasting
tax_rate = float(income.loc[base_year, 'Tax Rate For Calcs'])
rev0 = float(income.loc[base_year, 'Total Revenue'])
ebit0 = float(income.loc[base_year, 'EBIT'])
da0 = float(cashflow.loc[base_year, 'Depreciation'])
capex0 = float(cashflow.loc[base_year, 'Capital Expenditure'])

# Working capital in the FCF formula typically refers to *operating* NWC,
# excluding cash and financing items.
current_assets = float(balance.loc[base_year, 'Current Assets'])
current_liabilities = float(balance.loc[base_year, 'Current Liabilities'])
cash_like = float(balance.loc[base_year, 'Cash Cash Equivalents And Short Term Investments'])
current_debt = float(balance.loc[base_year, 'Current Debt'])
op_wc0 = (current_assets - cash_like) - (current_liabilities - current_debt)

ebit_margin = ebit0 / rev0
da_ratio = da0 / rev0
capex_ratio = (-capex0) / rev0
op_wc_ratio = op_wc0 / rev0

# Simple revenue-growth assumptions (edit freely)
forecast_years = [2026, 2027, 2028, 2029, 2030]
revenue_growth = [0.07, 0.06, 0.05, 0.04, 0.035]

revenue = [rev0]
for g in revenue_growth:
    revenue.append(revenue[-1] * (1 + g))
revenue = np.array(revenue[1:])

proj = pd.DataFrame({'Year': forecast_years, 'Revenue': revenue})
proj['EBIT'] = proj['Revenue'] * ebit_margin
proj['NOPAT'] = proj['EBIT'] * (1 - tax_rate)
proj['D&A'] = proj['Revenue'] * da_ratio
proj['CapEx'] = proj['Revenue'] * capex_ratio
proj['ΔWC'] = (proj['Revenue'].diff().fillna(proj['Revenue'].iloc[0] - rev0)) * op_wc_ratio
proj['FCF'] = proj['NOPAT'] + proj['D&A'] - proj['CapEx'] - proj['ΔWC']

proj_view = proj.assign(
    **{
        'Revenue ($B)': proj['Revenue'] / 1e9,
        'ΔWC ($B)': proj['ΔWC'] / 1e9,
        'FCF ($B)': proj['FCF'] / 1e9,
    }
)[['Year', 'Revenue ($B)', 'ΔWC ($B)', 'FCF ($B)']]
proj_view.round(2)


,Year,Revenue ($B),ΔWC ($B),FCF ($B)
0,2026,101.46,-0.39,2.23
1,2027,107.55,-0.36,2.31
2,2028,112.93,-0.32,2.36
3,2029,117.45,-0.26,2.40
4,2030,121.56,-0.24,2.45


In [4]:
# WACC (teaching assumptions)
price = pd.read_excel(DATA_DIR / 'tesla_daily_price.xlsx', index_col=0)
latest_close = float(price['Close'].iloc[-1])
shares = float(balance.loc[base_year, 'Ordinary Shares Number'])
market_cap = shares * latest_close

total_debt = float(balance.loc[base_year, 'Total Debt'])
cash_eq = float(balance.loc[base_year, 'Cash And Cash Equivalents'])
net_debt = total_debt - cash_eq
V = market_cap + total_debt

rf = 0.04
beta = 2.0
erp = 0.05
re = rf + beta * erp
rd_pre_tax = 0.06
rd = rd_pre_tax * (1 - tax_rate)
wacc = re * (market_cap / V) + rd * (total_debt / V)

wacc


0.13917642097924557

In [5]:
# Terminal growth (cap at ~4%)
equity = balance['Common Stock Equity'].dropna().sort_index()
ni = income['Net Income From Continuing Operation Net Minority Interest'].dropna().sort_index()
roe_2025 = float(ni.loc[base_year] / ((equity.loc[base_year] + equity.loc[prev_year]) / 2))
g_terminal = float(np.clip(roe_2025, 0.0, 0.04))

g_terminal


0.04

In [6]:
# Baseline valuation
fcf = proj['FCF'].to_numpy()
tv = terminal_value_growing_perpetuity(fcf_t=fcf[-1], wacc=wacc, g=g_terminal)
enterprise_value = dcf_pv(fcf, wacc) + tv / (1 + wacc) ** len(fcf)
equity_value = enterprise_value - net_debt
intrinsic_price = equity_value / shares

{'EV': enterprise_value, 'Equity value': equity_value, 'USD/share': intrinsic_price}


{'EV': np.float64(21409121996.47635),
 'Equity value': np.float64(23203121996.47635),
 'USD/share': np.float64(6.185849639156585)}

In [7]:
# Carbon adjustment (illustrative): carbon cost = (revenue * intensity) * carbon price
ei = 12.0  # tCO2e per $1M revenue (Scope 1+2 proxy)
carbon_price = 100.0  # $/tCO2e

revenue_musd = proj['Revenue'] / 1e6
carbon_cost = revenue_musd * ei * carbon_price
fcf_carbon = fcf - carbon_cost.to_numpy() * (1 - tax_rate)

enterprise_value_carbon = dcf_pv(fcf_carbon, wacc) + terminal_value_growing_perpetuity(
    fcf_t=fcf_carbon[-1], wacc=wacc, g=g_terminal
) / (1 + wacc) ** len(fcf_carbon)
equity_value_carbon = enterprise_value_carbon - net_debt
price_carbon = equity_value_carbon / shares

{'USD/share (baseline)': intrinsic_price, 'USD/share (carbon-adjusted)': price_carbon}


{'USD/share (baseline)': np.float64(6.185849639156585),
 'USD/share (carbon-adjusted)': np.float64(5.941597888899622)}

## Page 23 - Sensitivity Analysis

Sensitivity analysis asks: how much does intrinsic value change when we vary key assumptions such as:

- WACC
- terminal growth rate $g$
- (optionally) carbon price / emissions intensity

Use the slide screenshot for the sensitivity table layout.

<details>
<summary>Slide screenshot (PDF page 23)</summary>

![](pic/0209_lecture6_firm_valuation/page-23.png)

</details>


## Page 24 - Types of multiples

Common valuation multiples and when they are useful:

| Multiple | When it is useful |
|---|---|
| Enterprise value / sales (EV/Sales) | Helpful for high-growth or low-profit firms |
| Enterprise value / EBITDA (EV/EBITDA) | Popular in cyclical industries; cross-country comparisons; less sensitive to leverage |
| Enterprise value / EBIT (EV/EBIT) | Useful for capital-intensive businesses |
| Price / earnings (P/E) | Sensitive to one-offs; forward P/E is widely used |
| Price / book (P/B) | Often used for financial institutions |
| Enterprise value / total assets (EV/Assets) | Useful for utilities and other fixed-asset-heavy firms |

*Source in slide: J.P. Morgan.*

<details>
<summary>Slide screenshot (PDF page 24)</summary>

![](pic/0209_lecture6_firm_valuation/page-24.png)

</details>


## Page 25 - Tesla Comparable Firms

The slide provides a peer set for **pure-play EV manufacturers**. Use it as a starting point for comparable-company
analysis (see screenshot).

<details>
<summary>Slide screenshot (PDF page 25)</summary>

![](pic/0209_lecture6_firm_valuation/page-25.png)

</details>


## Page 26 - Tesla Comparable Firms

The slide provides a peer set for **technology / autonomy-adjacent firms**. These peers can be useful when you
want to cross-check EV makers against broader tech valuations (see screenshot).

<details>
<summary>Slide screenshot (PDF page 26)</summary>

![](pic/0209_lecture6_firm_valuation/page-26.png)

</details>


## Page 27 - Comparable Company Analysis

Comparable-company analysis typically:

1. selects a peer set
2. computes valuation multiples (EV/Sales, EV/EBITDA, P/E, etc.)
3. applies peer multiples to the target's metrics to infer an implied valuation range

See the slide screenshot for the template.

<details>
<summary>Slide screenshot (PDF page 27)</summary>

![](pic/0209_lecture6_firm_valuation/page-27.png)

</details>


### Comparable-company practice (optional)

The slide suggests replacing the template with Tesla's comparables.
Below is a compact `akshare` workflow for a handful of peer valuation metrics.

Tip: this section is **optional** and may require network access. Set `RUN_ONLINE = True` (or export `RUN_ONLINE=1`)
if you want to fetch live market data.


In [8]:
try:
    import akshare as ak
except ModuleNotFoundError:
    ak = None
    print('akshare is not installed; skipping comparable-company practice (optional).')

import time

PEERS = ['TSLA', 'GM', 'F', 'TM', 'NVDA', 'AAPL', 'GOOGL']

CACHE_PATH = DATA_DIR / 'peer_snapshot.csv'

# Baidu valuation indicators (ak.stock_us_valuation_baidu)
IND_MCAP = '总市值'
IND_PE = '市盈率(TTM)'
IND_PB = '市净率'
IND_PCF = '市现率'
PERIOD = '近一年'
ANNUAL = '年报'

def with_retry(fn, retries=3, backoff=0.6):
    last = None
    for k in range(retries):
        try:
            return fn()
        except Exception as e:
            last = e
            time.sleep(backoff * (k + 1))
    raise last

def latest_value(df):
    # akshare returns columns: date, value
    return float(df.iloc[-1]['value'])

if ak is None:
    RUN_ONLINE = False

rows = []
peer_snapshot = pd.DataFrame()
if RUN_ONLINE:
    for tkr in PEERS:
        try:
            mcap_yi = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_MCAP, PERIOD)))
            pe_ttm = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PE, PERIOD)))
            pb = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PB, PERIOD)))
            pcf = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PCF, PERIOD)))

            # Unit note: Baidu reports 总市值 in '亿' (1e8). Convert to USD dollars.
            market_cap = mcap_yi * 1e8

            # Latest close from Sina (ak.stock_us_daily)
            px = with_retry(lambda: ak.stock_us_daily(symbol=tkr, adjust=''))
            close = float(px.iloc[-1]['close'])

            # Revenue (latest annual) from Eastmoney indicator table
            fin = with_retry(lambda: ak.stock_financial_us_analysis_indicator_em(symbol=tkr, indicator=ANNUAL))
            fin_latest = fin.sort_values('REPORT_DATE').iloc[-1]
            revenue = float(fin_latest['OPERATE_INCOME'])
            currency = str(fin_latest.get('CURRENCY_ABBR', ''))

            mcap_to_rev = (market_cap / revenue) if currency == 'USD' and revenue else np.nan

            rows.append(
                {
                    'Ticker': tkr,
                    'Close': close,
                    'Market cap ($)': market_cap,
                    'P/E (TTM)': pe_ttm,
                    'P/B': pb,
                    'P/CF': pcf,
                    'Revenue (latest)': revenue,
                    'Currency': currency,
                    'MktCap/Revenue (USD only)': mcap_to_rev,
                }
            )
        except Exception as e:
            rows.append({'Ticker': tkr, 'Error': f'{type(e).__name__}: {e}'})
    peer_snapshot = pd.DataFrame(rows).set_index('Ticker') if rows else pd.DataFrame()
    if not peer_snapshot.empty:
        peer_snapshot.to_csv(CACHE_PATH)
        print(f'Cached peer snapshot to: {CACHE_PATH.resolve()}')
else:
    if CACHE_PATH.exists():
        peer_snapshot = pd.read_csv(CACHE_PATH, index_col=0)
        print(f'Loaded cached peer snapshot from: {CACHE_PATH.resolve()}')
    else:
        print('Skipping akshare fetch (RUN_ONLINE=False) and no cache found.')

peer_snapshot


Loaded cached peer snapshot from: C:\Users\Jinquan Ye\OneDrive - Duke University\Research\teaching\K10_calculation\materials\data\peer_snapshot.csv


,Close,Market cap ($),P/E (TTM),P/B,P/CF,Revenue (latest),Currency,MktCap/Revenue (USD only)
Ticker,,,,,,,,
TSLA,411.11,1.542662e+12,406.60,18.77,98.35,9.482700e+10,USD,16.268173
GM,84.24,7.615000e+10,28.24,1.25,3.05,1.850190e+11,USD,0.411579
F,13.80,5.498600e+10,11.68,1.22,2.73,1.849920e+11,USD,0.297234
TM,244.22,3.183010e+11,12.71,1.28,10.23,4.803670e+13,JPY,NaN
NVDA,185.41,4.505463e+12,45.42,37.90,54.59,1.304970e+11,USD,34.525414
AAPL,278.12,4.083119e+12,34.67,46.37,30.41,4.161610e+11,USD,9.811393
GOOGL,322.86,3.905637e+12,29.55,9.40,23.97,4.028360e+11,USD,9.695352


In [9]:
from IPython.display import display

if peer_snapshot.empty or peer_snapshot.dropna(how='all').empty:
    print('No online data to summarize. Set RUN_ONLINE=True to fetch via akshare.')
else:
    numeric = peer_snapshot.select_dtypes(include='number')
    if numeric.empty:
        print('No numeric columns to summarize (all fetches failed).')
    else:
        summary = numeric.describe(percentiles=[0.5]).loc[['mean', '50%']]
        display(summary)


,Close,Market cap ($),P/E (TTM),P/B,P/CF,Revenue (latest),MktCap/Revenue (USD only)
mean,219.965714,2.069474e+12,81.267143,16.598571,31.904286,7.064434e+12,11.834858
50%,244.220000,1.542662e+12,29.550000,9.400000,23.970000,1.850190e+11,9.753373
